In [2]:
from datetime import timedelta, datetime
import rsa_headers as rh
import rsa_data as rd
import main
import config
import uuid
from pandasql import sqldf
from sqlalchemy.dialects.postgresql import insert

files = r"C:\Users\MB27{}5851\Desktop\Temp\rsa_traffic_counts\SMEC RSA Files_GP PRM Sites_Dec21toFeb22\{}337_2{}22{}131.RSA"
import pandas as pd
pd.set_option("display.max_columns", None)

testfile = r"C:\Users\MB27{}5851\Desktop\Temp\SYNTELL-test.csv"

In [4]:
QRY = """select * from trafc.electronic_count_data_type_10 t10 where exists (select header_id from trafc.electronic_count_header h where h.adt_total is null
and h.header_id = t10.header_id)"""
QRY_TEST =  """select * from trafc.electronic_count_data_type_10 t10 where site_id = '0288' and year = 2021"""

{HEADER_COLUMNS_QRY = """select * from trafc.electronic_count_header limit 1"""}

In [16]:
header_columns = list(pd.read_sql_query(HEADER_COLUMNS_QRY,config.ENGINE).columns)

In [18]:
data = pd.read_sql_query(QRY_TEST, config.ENGINE)

In [21]:
df.columns

Index(['id', 'site_id', 'header_id', 'year',
       'number_of_fields_associated_with_the_basic_vehicle_data',
       'data_source_code', 'edit_code', 'departure_date', 'departure_time',
       'assigned_lane_number', 'physical_lane_number', 'forward_reverse_code',
       'vehicle_category', 'vehicle_class_code_primary_scheme',
       'vehicle_class_code_secondary_scheme', 'vehicle_speed',
       'vehicle_length', 'site_occupancy_time_in_milliseconds',
       'chassis_height_code', 'vehicle_following_code', 'vehicle_tag_code',
       'trailer_count', 'axle_count', 'bumper_to_1st_axle_spacing',
       'tyre_type', 'start_datetime', 'direction', 'data_id'],
      dtype='object')

In [115]:
((data.loc[data['vehicle_speed']>speed_limit].count()[0]/data.count()[0])*100).round(2)

43.74

In [98]:
def header_update(data):
	speed_limit_qry = f"select max_speed from trafc.countstation where tcname = '{data['site_id'][0]}' ;"
	speed_limit = pd.read_sql_query(speed_limit_qry,config.ENGINE)
	speed_limit = speed_limit['max_speed'][0]

	UPDATE_STRING = f"""update
		trafc.electronic_count_header
	set
		total_light_positive_direction = {data.loc[(data['vehicle_class_code_secondary_scheme']<=1)&(data['direction']=='N')].count()[0].round().astype(int)},
		total_light_negative_direction = {data.loc[(data['vehicle_class_code_secondary_scheme']<=1)&(data['direction']=='P')].count()[0].round().astype(int)},
		total_light_vehicles = {data.loc[data['vehicle_class_code_secondary_scheme']<=1].count()[0].round().astype(int)},
		total_heavy_positive_direction = {data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].count()[0].round().astype(int)},
		total_heavy_negative_direction = {data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].count()[0].round().astype(int)},
		total_heavy_vehicles = {data.loc[data['vehicle_class_code_secondary_scheme']>1].count()[0].round().astype(int)},
		total_short_heavy_positive_direction = {data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='N')].count()[0].round().astype(int)},
		total_short_heavy_negative_direction = {data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='P')].count()[0].round().astype(int)},
		total_short_heavy_vehicles = {data.loc[data['vehicle_class_code_secondary_scheme']==2].count()[0].round().astype(int)},
		total_medium_heavy_positive_direction = {data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='N')].count()[0].round().astype(int)},
		total_medium_heavy_negative_direction = {data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='P')].count()[0].round().astype(int)},
		total_medium_heavy_vehicles = {data.loc[data['vehicle_class_code_secondary_scheme']==3].count()[0].round().astype(int)},
		total_long_heavy_positive_direction = {data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='N')].count()[0].round().astype(int)},
		total_long_heavy_negative_direction = {data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='P')].count()[0].round().astype(int)},
		total_long_heavy_vehicles = {data.loc[data['vehicle_class_code_secondary_scheme']==4].count()[0].round().astype(int)},
		total_vehicles_positive_direction = {data.loc[data['direction']=='N'].count()[0].round().astype(int)},
		total_vehicles_negative_direction = {data.loc[data['direction']=='P'].count()[0].round().astype(int)},
		total_vehicles = {data.count()[0]},
		average_speed_positive_direction = {data.loc[data['direction']=='N']['vehicle_speed'].mean().round(2)},
		average_speed_negative_direction = {data.loc[data['direction']=='P']['vehicle_speed'].mean().round(2)},
		average_speed = {data['vehicle_speed'].mean().round(2)},
		average_speed_light_vehicles_positive_direction = {data['vehicle_speed'].loc[(data['vehicle_class_code_secondary_scheme']<=1)&(data['direction']=='N')].mean().round(2)},
		average_speed_light_vehicles_negative_direction = {data['vehicle_speed'].loc[(data['vehicle_class_code_secondary_scheme']<=1)&(data['direction']=='P')].mean().round(2)},
		average_speed_light_vehicles = {data['vehicle_speed'].loc[data['vehicle_class_code_secondary_scheme']<=1].mean().round(2)},
		average_speed_heavy_vehicles_positive_direction = {data['vehicle_speed'].loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].mean().round(2)},
		average_speed_heavy_vehicles_negative_direction = {data['vehicle_speed'].loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].mean().round(2)},
		average_speed_heavy_vehicles = {data['vehicle_speed'].loc[data['vehicle_class_code_secondary_scheme']>1].mean().round(2)},
		truck_split_positive_direction = '{str((((data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='N')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='P')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='N')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].count())[0])*100).round().astype(int))}',
		truck_split_negative_direction = '{str((((data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='P')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='N')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='P')].count()/data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].count())[0])*100).round().astype(int))}',
		truck_split_total = '{str((((data.loc[data['vehicle_class_code_secondary_scheme']==2].count()/data.loc[data['vehicle_class_code_secondary_scheme']>1].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[data['vehicle_class_code_secondary_scheme']==3].count()/data.loc[data['vehicle_class_code_secondary_scheme']>1].count())[0])*100).round().astype(int)) +":"+ str((((data.loc[data['vehicle_class_code_secondary_scheme']==4].count()/data.loc[data['vehicle_class_code_secondary_scheme']>1].count())[0])*100).round().astype(int))}',
		estimated_axles_per_truck_negative_direction = {((data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='N')].count()[0]*2+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='N')].count()[0]*5+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='N')].count()[0]*7)/(data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='N')].count()[0]+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='N')].count()[0]+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='N')].count()[0])).round(2)},
		estimated_axles_per_truck_positive_direction = {((data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='P')].count()[0]*2+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='P')].count()[0]*5+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='P')].count()[0]*7)/(data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='P')].count()[0]+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='P')].count()[0]+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='P')].count()[0])).round(2)},
		estimated_axles_per_truck_total = {((data.loc[data['vehicle_class_code_secondary_scheme']==2].count()[0]*2+data.loc[data['vehicle_class_code_secondary_scheme']==3].count()[0]*5+data.loc[data['vehicle_class_code_secondary_scheme']==4].count()[0]*7)/(data.loc[data['vehicle_class_code_secondary_scheme']==2].count()[0]+data.loc[data['vehicle_class_code_secondary_scheme']==3].count()[0]+data.loc[data['vehicle_class_code_secondary_scheme']==4].count()[0])).round(2)},
		percentage_speeding_positive_direction = {((data.loc[(data['vehicle_speed']>speed_limit)&(data['direction']=='P')].count()[0]/data.loc[data['direction'=='P']].count()[0])*100).round(2)},
		percentage_speeding_negative_direction = {((data.loc[(data['vehicle_speed']>speed_limit)&(data['direction']=='N')].count()[0]/data.loc[data['direction'=='N']].count()[0])*100).round(2)},
		percentage_speeding_total = {((data.loc[data['vehicle_speed']>speed_limit].count()[0]/data.count()[0])*100).round(2)},
		vehicles_with_rear_to_rear_headway_less_than_2sec_positive_dire = {data.loc[(data['vehicle_following_code']==2)&data['direction']=='N'].count()[0]},
		vehicles_with_rear_to_rear_headway_less_than_2sec_negative_dire = {data.loc[(data['vehicle_following_code']==2)&data['direction']=='P'].count()[0]},
		vehicles_with_rear_to_rear_headway_less_than_2sec_total = {data.loc[data['vehicle_following_code']==2].count()[0]},
		estimated_e80_positive_direction = {data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='N')].count()[0]*0.6+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='N')].count()[0]*2.5+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='N')].count()[0]*2.1},
		estimated_e80_negative_direction = {data.loc[(data['vehicle_class_code_secondary_scheme']==2)&(data['direction']=='P')].count()[0]*0.6+data.loc[(data['vehicle_class_code_secondary_scheme']==3)&(data['direction']=='P')].count()[0]*2.5+data.loc[(data['vehicle_class_code_secondary_scheme']==4)&(data['direction']=='P')].count()[0]*2.1},
		estimated_e80_on_road = {data.loc[data['vehicle_class_code_secondary_scheme']==2].count()[0]*0.6+data.loc[data['vehicle_class_code_secondary_scheme']==3].count()[0]*2.5+data.loc[data['vehicle_class_code_secondary_scheme']==4].count()[0]*2.1},
		adt_positive_direction = {data.loc[data['direction']=='N'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)},
		adt_negative_direction = {data.loc[data['direction']=='P'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)},
		adt_total = {data.groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)},
		adtt_positive_direction = {data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='N')].groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)},
		adtt_negative_direction = {data.loc[(data['vehicle_class_code_secondary_scheme']>1)&(data['direction']=='P')].groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)},
		adtt_total = {data.loc[data['vehicle_class_code_secondary_scheme']>1].groupby(pd.Grouper(key='start_datetime',freq='D')).count().mean()[0].round().astype(int)},
		highest_volume_per_hour_positive_direction = {data.loc[data['direction']=='N'].groupby(pd.Grouper(key='start_datetime',freq='H')).count().max()[0]},
		highest_volume_per_hour_negative_direction = {data.loc[data['direction']=='P'].groupby(pd.Grouper(key='start_datetime',freq='H')).count().max()[0]},
		highest_volume_per_hour_total = {data.groupby(pd.Grouper(key='start_datetime',freq='H')).count().max()[0]},
		"15th_highest_volume_per_hour_positive_direction" = {data.loc[data['direction']=='N'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.15)[0].astype(int)},
		"15th_highest_volume_per_hour_negative_direction" = {data.loc[data['direction']=='P'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.15)[0].astype(int)},
		"15th_highest_volume_per_hour_total" = {data.groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.15)[0].astype(int)},
		"30th_highest_volume_per_hour_positive_direction" = {data.loc[data['direction']=='N'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.30)[0].astype(int)},
		"30th_highest_volume_per_hour_negative_direction" = {data.loc[data['direction']=='P'].groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.30)[0].astype(int)},
		"30th_highest_volume_per_hour_total" = {data.groupby(pd.Grouper(key='start_datetime',freq='D')).count().quantile(0.30)[0].astype(int)},
		"15th_percentile_speed_positive_direction" = {data.loc[data['direction']=='N']['vehicle_speed'].quantile(0.15).round(2)},
		"15th_percentile_speed_negative_direction" = {data.loc[data['direction']=='P']['vehicle_speed'].quantile(0.15).round(2)},
		"15th_percentile_speed_total" = {data['vehicle_speed'].quantile(0.15).round(2)},
		"85th_percentile_speed_positive_direction" = {data.loc[data['direction']=='N']['vehicle_speed'].quantile(0.85).round(2)},
		"85th_percentile_speed_negative_direction" = {data.loc[data['direction']=='P']['vehicle_speed'].quantile(0.85).round(2)},
		"85th_percentile_speed_total" = {data['vehicle_speed'].quantile(0.85).round(2)},
		avg_weekday_traffic = {data.groupby(pd.Grouper(key='start_datetime',freq='B')).count().mean()[0].round().astype(int)},
		number_of_days_counted = {data.groupby([data['start_datetime'].dt.to_period('D')]).count().count()[0]},
		duration_hours = {data.groupby([data['start_datetime'].dt.to_period('H')]).count().count()[0]}
	where
		site_id = '{data['site_id'][0]}'
		and start_datetime = '{data['start_datetime'][0]}'
		and end_datetime = '{data['end_datetime'][0]}';
	"""
	return UPDATE_STRING

SyntaxError: f-string: empty expression not allowed (4249578818.py, line 74)

In [64]:
def header_calcs(header, data, dtype):
    data['start_datetime'] = pd.to_datetime(data['start_datetime'])
    if dtype == 21:
        header['adt_total'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['header_id']]).sum().mean()
        header['adt_positive_direction'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().mean()
        header['adt_positive_direction'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().mean()

        header['adtt_total'] = data['total_heavy_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['header_id']]).sum().mean()
        header['adtt_positive_direction'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().mean()
        header['adtt_positive_direction'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().mean()

        header['total_vehicles'] = data['total_vehicles_type21'].groupby(data['header_id']).sum()[{}]
        header['total_vehicles_positive_direction'] = data['total_vehicles_type21'].groupby(data['direction'].loc[data['direction'] == 'N']).sum()[{}]
        header['total_vehicles_positive_direction'] = data['total_vehicles_type21'].groupby(data['direction'].loc[data['direction'] == 'N']).sum()[{}]

        header['total_heavy_vehicles'] = data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[{}]
        header['total_heavy_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]
        header['total_heavy_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]
        header['truck_split_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}] / data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[{}]
        header['truck_split_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}] / data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[{}]

        header['total_light_vehicles'] = data['total_light_vehicles_type21'].groupby(data['header_id']).sum()[{}]
        header['total_light_positive_direction'] = data['total_light_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]
        header['total_light_positive_direction'] = data['total_light_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]

        header['total_short_heavy_vehicles'] = data['short_heavy_vehicles'].groupby(data['header_id']).sum()[{}]
        header['total_short_heavy_positive_direction'] = data['short_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]
        header['total_short_heavy_positive_direction'] = data['short_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]

        header['total_medium_heavy_vehicles'] = data['medium_heavy_vehicles'].groupby(data['header_id']).sum()[{}]
        header['total_medium_heavy_positive_direction'] = data['medium_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]
        header['total_medium_heavy_positive_direction'] = data['medium_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]

        header['total_long_heavy_vehicles'] = data['long_heavy_vehicles'].groupby(data['header_id']).sum()[{}]
        header['total_long_heavy_positive_direction'] = data['long_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]
        header['total_long_heavy_positive_direction'] = data['long_heavy_vehicles'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]

        header['vehicles_with_rear_to_rear_headway_less_than_2sec_positive_dire'] = data['rear_to_rear_headway_shorter_than_2_seconds'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]
        header['vehicles_with_rear_to_rear_headway_less_than_2sec_negative_dire'] = data['rear_to_rear_headway_shorter_than_2_seconds'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]
        header['vehicles_with_rear_to_rear_headway_less_than_2sec_total'] = data['rear_to_rear_headway_shorter_than_2_seconds'].groupby(data['header_id']).sum()[{}]
    
        header['type_21_count_interval_minutes'] = data['duration_min'].mean()

        header['highest_volume_per_hour_positive_direction'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().max()
        header['highest_volume_per_hour_negative_direction'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().max()
        header['highest_volume_per_hour_total'] = data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['header_id']]).sum().max()

        header['15th_highest_volume_per_hour_positive_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().quantile(q={}.15,  interpolation='linear'))
        header['15th_highest_volume_per_hour_negative_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().quantile(q={}.15,  interpolation='linear'))
        header['15th_highest_volume_per_hour_total'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['header_id']]).sum().quantile(q={}.15, interpolation='linear'))
        
        header['3{}th_highest_volume_per_hour_positive_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().quantile(q={}.3,  interpolation='linear'))
        header['3{}th_highest_volume_per_hour_negative_direction'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().quantile(q={}.3, interpolation='linear'))
        header['3{}th_highest_volume_per_hour_total'] = round(data['total_vehicles_type21'].groupby([data['start_datetime'].dt.to_period('H'), data['header_id']]).sum().quantile(q={}.3, interpolation='linear'))

        # header['average_speed_positive_direction'] = 
        # header['average_speed_negative_direction'] = 
        header['average_speed'] = ((
            (header['speedbin1'] * data['speedbin1'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin2'] * data['speedbin2'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin3'] * data['speedbin3'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin4'] * data['speedbin4'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin5'] * data['speedbin5'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin6'] * data['speedbin6'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin7'] * data['speedbin7'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin8'] * data['speedbin8'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin9'] * data['speedbin9'].groupby(data['header_id']).sum()[{}]) 
            )
            / data['sum_of_heavy_vehicle_speeds'].groupby(data['header_id']).sum()[{}]
            )
        # header['average_speed_light_vehicles_positive_direction'] = 
        # header['average_speed_light_vehicles_negative_direction'] = 
        header['average_speed_light_vehicles'] = ((
            (header['speedbin1'] * data['speedbin1'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin2'] * data['speedbin2'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin3'] * data['speedbin3'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin4'] * data['speedbin4'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin5'] * data['speedbin5'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin6'] * data['speedbin6'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin7'] * data['speedbin7'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin8'] * data['speedbin8'].groupby(data['header_id']).sum()[{}]) +
            (header['speedbin9'] * data['speedbin9'].groupby(data['header_id']).sum()[{}]) -
            data['sum_of_heavy_vehicle_speeds'].groupby(data['header_id']).sum()[{}]
            )
            / data['sum_of_heavy_vehicle_speeds'].groupby(data['header_id']).sum()[{}]
            )
        
        # header['average_speed_heavy_vehicles_positive_direction'] = 
        # header['average_speed_heavy_vehicles_negative_direction'] = 
        # header['average_speed_heavy_vehicles'] = 

        header['truck_split_positive_direction'] = (str(round(data['short_heavy_vehicles'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[{}] / 
        data['total_heavy_vehicles_type21'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[{}]*1{}{})) + ' : ' +
        str(round(data['medium_heavy_vehicles'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[{}] / 
        data['total_heavy_vehicles_type21'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[{}]*1{}{})) + ' : ' +
        str(round(data['long_heavy_vehicles'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[{}] / 
        data['total_heavy_vehicles_type21'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[{}]*1{}{}))
        )
        header['truck_split_negative_direction'] = (str(round(data['short_heavy_vehicles'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[{}] / 
        data['total_heavy_vehicles_type21'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[{}]*1{}{})) + ' : ' +
        str(round(data['medium_heavy_vehicles'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[{}] / 
        data['total_heavy_vehicles_type21'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[{}]*1{}{})) + ' : ' +
        str(round(data['long_heavy_vehicles'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[{}] / 
        data['total_heavy_vehicles_type21'].groupby([data['header_id'], data['direction'].loc[data['direction'] == 'N']]).sum()[{}]*1{}{}))
        )
        header['truck_split_total'] = (str(round(data['short_heavy_vehicles'].groupby(data['header_id']).sum()[{}] / 
        data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[{}]*1{}{})) + ' : ' +
        str(round(data['medium_heavy_vehicles'].groupby(data['header_id']).sum()[{}] / 
        data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[{}]*1{}{})) + ' : ' +
        str(round(data['long_heavy_vehicles'].groupby(data['header_id']).sum()[{}] / 
        data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[{}]*1{}{}))
        )

        return header
    elif dtype == 3{}:
        if header['adt_total'].isnull().all():
            header['adt_total'] = data['total_vehicles_type3{}'].groupby([data['start_datetime'].dt.to_period('D'), data['header_id']]).sum().mean()
            header['adt_positive_direction'] = data['total_vehicles_type3{}'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().mean()
            header['adt_positive_direction'] = data['total_vehicles_type3{}'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().mean()
        else:
            pass

        if header['adtt_total'].isnull().all():
            header['adtt_total'] = data['total_heavy_vehicles_type3{}'].groupby([data['start_datetime'].dt.to_period('D'), data['header_id']]).sum().mean()
            header['adtt_positive_direction'] = data['total_vehicles_type3{}'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().mean()
            header['adtt_positive_direction'] = data['total_vehicles_type3{}'].groupby([data['start_datetime'].dt.to_period('D'), data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum().mean()
        else:
            pass

        if header['total_vehicles'].isnull().all():
            header['total_vehicles'] = data['total_vehicles_type3{}'].groupby(data['header_id']).sum()[{}]
            header['total_vehicles_positive_direction'] = data['total_vehicles_type3{}'].groupby(data['direction'].loc[data['direction'] == 'N']).sum()[{}]
            header['total_vehicles_positive_direction'] = data['total_vehicles_type3{}'].groupby(data['direction'].loc[data['direction'] == 'N']).sum()[{}]
        else:
            pass
        
        if header['total_heavy_vehicles'].isnull().all():
            header['total_heavy_vehicles'] = data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[{}]
            header['total_heavy_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]
            header['total_heavy_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]
            header['truck_split_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}] / data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[{}]
            header['truck_split_positive_direction'] = data['total_heavy_vehicles_type21'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}] / data['total_heavy_vehicles_type21'].groupby(data['header_id']).sum()[{}]
        else:
            pass

        if header['total_light_vehicles'].isnull().all():
            header['total_light_vehicles'] = data['total_light_vehicles_type3{}'].groupby(data['header_id']).sum()[{}]
            header['total_light_positive_direction'] = data['total_light_vehicles_type3{}'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]
            header['total_light_positive_direction'] = data['total_light_vehicles_type3{}'].groupby([data['direction'].loc[data['direction'] == 'N'], data['header_id']]).sum()[{}]
        else:
            pass

        return header

    elif dtype == 7{}:
    
        return header

    elif dtype == 1{}:
    
        return header

    elif dtype == 6{}:
        
        return header

    else:
        return header

In [65]:
header_out = header_calcs(header, data, 21)

In [66]:
header_out = header_calcs(header_out, data, 3{})

In [100]:
def psql_insert_copy(table, conn, keys, data_iter):
    """
    Execute SQL statement inserting data

    Parameters
    ----------
    table : pandas.io.sql.SQLTable
    conn : sqlalchemy.engine.Engine or sqlalchemy.engine.Connection
    keys : list of str
        Column names
    data_iter : Iterable that iterates the values to be inserted
    """
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek({})

        columns = ", ".join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = "{}.{}".format(table.schema, table.name)
        else:
            table_name = table.name

        sql = "COPY {} ({}) FROM STDIN WITH CSV".format(table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

In [101]:
def push_to_partitioned_table(df, table, subset) -> None:
    yr = data['year'].unique()[{}]
    print(yr)
    try:
        df.to_sql(
            table+'_'+str(yr),
            con=config.ENGINE,
            schema="trafc",
            if_exists="append",
            index=False,
            method=psql_insert_copy,
        )
    except Exception:
        df = df.drop_duplicates(subset=subset)
        df.to_sql(
            table+'_'+yr,
            con=config.ENGINE,
            schema="trafc",
            if_exists="append",
            index=False,
            method=psql_insert_copy,
        )

def push_to_db(df, table, subset) -> None:
    try:
        df.to_sql(
            table,
            con=config.ENGINE,
            schema="trafc",
            if_exists="append",
            index=False,
            method=psql_insert_copy,
        )
    except Exception:
        df = df.drop_duplicates(subset=subset)
        df.to_sql(
            table,
            con=config.ENGINE,
            schema="trafc",
            if_exists="append",
            index=False,
            method=psql_insert_copy,
        )


In [42]:
def postgres_upsert(table, conn, keys, data_iter):
    data = [dict(zip(keys, row)) for row in data_iter]

    insert_statement = insert(table.table).values(data)
    upsert_statement = insert_statement.on_conflict_do_update(
        constraint=f"{table.table.name}_un",
        set_={c.key: c for c in insert_statement.excluded},
    )
    conn.execute(upsert_statement)

In [98]:
def postgres_upsert_1(table, conn, keys, data_iter):
    data = [dict(zip(keys, row)) for row in data_iter]

    insert_statement = insert(table.table).values(data)
    upsert_statement = insert_statement.on_conflict_do_update(
        constraint=f"{table.table.name}_un",
        set_={c.key: c for c in insert_statement.excluded},
    )
    conn.execute(upsert_statement)

In [ ]:
insert_statement = insert(table.table).values(data)
upsert_statement = insert_statement.on_conflict_do_update(
    constraint=f"{table.table.name}_un",
    set_={c.key: c for c in insert_statement.excluded else },
)
conn.execute(upsert_statement)

In [102]:
push_to_partitioned_table(
            data,
            "electronic_count_data_partitioned",
            ["site_id", "start_datetime", "lane_number"],
        )

KeyError: 0

In [69]:
push_to_db(
    header_out,
    "electronic_count_header",
    ["site_id", "start_datetime", "end_datetime"],
)

In [142]:
data['year'].unique()[{}]

2022

In [82]:
data.to_csv(r"C:\Users\MB27{}5851\Desktop\Temp\rsa_traffic_counts\1.csv")